In [ ]:
import boto3


## User 2 has to add the ARN value of the role created by user-1

Role_Arn = 'arn:aws:iam::odh1:role/S3Access'


odh2_user1_sts = boto3.client('sts',
           aws_access_key_id='odh2user1',
           aws_secret_access_key='odh2user1pass',
           endpoint_url='http://ceph-rgw.odh-1.svc.cluster.local',
           region_name='')


response = odh2_user1_sts.assume_role(
            RoleArn=Role_Arn,
            RoleSessionName='session_odh2user1',
            DurationSeconds=3600
            )


aws_access_key_id = response['Credentials']['AccessKeyId']
aws_secret_access_key = response['Credentials']['SecretAccessKey']
endpoint_url='http://ceph-rgw.odh-1.svc.cluster.local'

s3client = boto3.client('s3',
            aws_access_key_id = response['Credentials']['AccessKeyId'],
            aws_secret_access_key = response['Credentials']['SecretAccessKey'],
            aws_session_token = response['Credentials']['SessionToken'],
            endpoint_url='http://ceph-rgw.odh-1.svc.cluster.local',
            region_name='',)

bucket_name = 'ufo-dataset'
#s3bucket = s3client.create_bucket(Bucket=bucket_name)
#print(s3client.list_buckets())

print("Getting files from odh1user1 dataset "+ bucket_name +" bucket, without sharing credentials \n")
for key in s3client.list_objects(Bucket=bucket_name)['Contents']:
    print(key['Key'])
    
#print("\n Creating a new bucket from odh2user1 account")
#try:
#    s3bucket = s3client.create_bucket(Bucket='odh2user1-bucket')
#except ClientError as e:
#    print("\n Bucket already exists \n")

# Retrieve the list of existing buckets
#response = s3client.list_buckets()

# Output the bucket names
#print('Existing buckets:')
#for bucket in response['Buckets']:
#    print(f'  {bucket["Name"]}')

In [ ]:
import pyspark

from pyspark.context import SparkContext
from pyspark.sql import SparkSession, SQLContext

spark = SparkSession\
    .builder\
    .master("local")\
    .appName("UFO_Analysis_using_SPARK")\
    .getOrCreate()

In [ ]:
S3_SSL = 'False'

hadoopConf=spark.sparkContext._jsc.hadoopConfiguration()
hadoopConf.set("fs.s3a.access.key", aws_access_key_id)
hadoopConf.set("fs.s3a.secret.key", aws_secret_access_key)
hadoopConf.set("fs.s3a.endpoint", endpoint_url)
hadoopConf.set("fs.s3a.connection.ssl.enabled", S3_SSL)
hadoopConf.set("fs.s3a.path.style.access", "true")

In [ ]:
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

ceph_object_storage_dataset_path = "s3a://"+bucket_name+"/UFO_dataset_kaggle.csv"

df0 = spark.read.csv(ceph_object_storage_dataset_path, header=True)

In [ ]:
df0.printSchema()

## Business Qestion - 1 : What are the TOP-5 countries which reported UFO sighting

In [ ]:
plot1 = df0.groupBy("country").count().toPandas()
import plotly 
plotly.tools.set_credentials_file(username='xxx', api_key='xxxx')

import plotly.plotly as py
import plotly.graph_objs as go

data1 = [go.Bar(
    x=plot1['country'],
    y=plot1['count'],
    width = 0.8 
)]
py.iplot(data1, filename='basic-bar')


In [ ]:
from pyspark.sql import SparkSession, SQLContext
import os
import socket

# create a spark session
spark_cluster_url = f"spark://{os.environ['SPARK_CLUSTER']}:7077"
spark = SparkSession.builder.master(spark_cluster_url).getOrCreate()

# test your spark connection
spark.range(5, numPartitions=5).rdd.map(lambda x: socket.gethostname()).distinct().collect()

data = spark.read.csv('https://gitlab.com/opendatahub/opendatahub.io/-/raw/master/assets/files/tutorials/basic/sample_data.csv', sep=",", header=True)
df = data.toPandas()
df.head()
